In [1]:
!pip install efficientnet_pytorch


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=4846714f7f7b5e9c2458d8f94c2fc0b4ab3528f6f23529d

In [2]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [3]:
from google.colab import drive
drive.mount('/content/drive')

!unzip "/content/drive/MyDrive/Comys_Hackathon5.zip" -d "/content/Comys_Hackathon5"

Streaming output truncated to the last 5000 lines.
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Luc_Montagnier/distortion/Luc_Montagnier_0001_lowlight.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Luc_Montagnier/distortion/Luc_Montagnier_0001_foggy.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Luc_Montagnier/distortion/Luc_Montagnier_0001_blurred.jpg  
   creating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/Lucy_Liu_0005_sunny.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/Lucy_Liu_0005_resized.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/Lucy_Liu_0005_rainy.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/Lucy_Liu_0005_noisy.jpg  
  inflating

In [4]:
train_dir = '/content/Comys_Hackathon5/Comys_Hackathon5/Task_A/train'
val_dir = '/content/Comys_Hackathon5/Comys_Hackathon5/Task_A/val'


In [5]:
image_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [6]:
batch_size = 32

train_dataset = datasets.ImageFolder(root=train_dir, transform=image_transforms['train'])
val_dataset = datasets.ImageFolder(root=val_dir, transform=image_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes  # ['female', 'male']
print("Classes:", class_names)


Classes: ['female', 'male']


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EfficientNet.from_pretrained('efficientnet-b0')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)
model = model.to(device)


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 217MB/s]


Loaded pretrained weights for efficientnet-b0


In [8]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)


In [9]:
def evaluate(model, dataloader):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device).float()
            outputs = model(inputs)
            predictions = torch.sigmoid(outputs).cpu().numpy() > 0.5
            preds.extend(predictions.astype(int).flatten())
            labels.extend(targets.cpu().numpy().astype(int))

    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds)
    rec = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    return acc, prec, rec, f1

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device).float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"\nEpoch {epoch+1}/{epochs}")
    print(f"Train Loss: {running_loss/len(train_loader):.4f}")

    acc, prec, rec, f1 = evaluate(model, val_loader)
    print(f"Val → Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")



Epoch 1/5
Train Loss: 0.2566
Val → Acc: 0.9147, Prec: 0.9026, Rec: 0.9937, F1: 0.9459

Epoch 2/5
Train Loss: 0.0742
Val → Acc: 0.9242, Prec: 0.9083, Rec: 1.0000, F1: 0.9520

Epoch 3/5
Train Loss: 0.0460
Val → Acc: 0.9502, Prec: 0.9512, Rec: 0.9842, F1: 0.9674

Epoch 4/5
Train Loss: 0.0403
Val → Acc: 0.9052, Prec: 0.8880, Rec: 1.0000, F1: 0.9407

Epoch 5/5
Train Loss: 0.0553
Val → Acc: 0.9550, Prec: 0.9656, Rec: 0.9748, F1: 0.9702


In [10]:
torch.save(model.state_dict(), "efficientnet_gender_classifier.pth")


In [11]:
def test_model(test_path):
    test_dataset = datasets.ImageFolder(root=test_path, transform=image_transforms['val'])
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    acc, prec, rec, f1 = evaluate(model, test_loader)
    print(f"Test Results → Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1:.4f}")


In [12]:
test_model('/content/Comys_Hackathon5/Comys_Hackathon5/Task_A/val')


Test Results → Accuracy: 0.9550, Precision: 0.9656, Recall: 0.9748, F1-Score: 0.9702


In [13]:
!pip install torchviz efficientnet_pytorch


In [14]:

from torchviz import make_dot


model = EfficientNet.from_name('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, 1)


dummy_input = torch.randn(1, 3, 224, 224)


output = model(dummy_input)


make_dot(output, params=dict(model.named_parameters())).render("efficientnet_gender_diagram", format="png")


'efficientnet_gender_diagram.png'